In [2]:
import pyBach
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

<H1>Scraping Bachelor Episode and Reddit Data</H1>
pyBach is a collection of functions that allows for scraping of bachelor and bachelorette data from Wikipedia and Reddit
<ol>
    <li>First scrape last 5 season of The Bachelor and The Bachelorette into a pandas dataframe</li>
    <li>Join all individual dataframes into one large dataframe</li>

In [4]:
dfList = []
bachSeasons = range(19,24)
etteSeasons = range(10,15)

for bachSeason in bachSeasons:
    dfList.append(pyBach.scrapeWikiEpiTable(bachSeason))
for etteSeason in etteSeasons:
    dfList.append(pyBach.scrapeWikiEpiTable(etteSeason, show='bachelorette'))

df = pd.concat(dfList)

In [5]:
df = pyBach.appendStartTime(df) #default value is 6 days prior to episode airdate
df = pyBach.appendEndTime(df) #default value 0 days

In [6]:
subLists = []
for idx, row in df.iterrows():
    print(row['startEpoch'])
    print(row['endEpoch'])
    subLists.append(pyBach.getRedditSubsList(row['startEpoch'], row['endEpoch'])) #use my default PRAW User Agent
df['subList'] = subLists

1419933600
1420452000
1420538400
1421056800
1421143200
1421661600
1421748000
1422266400
1422352800
1422871200
1422957600
1423476000
1423476000
1423994400
1423562400
1424080800
1424167200
1424685600
1424772000
1425290400
1425373200
1425888000
1425373200
1425888000
1451383200
1451901600
1451988000
1452506400
1452592800
1453111200
1453197600
1453716000
1453802400
1454320800
1454407200
1454925600
1455012000
1455530400
1455616800
1456135200
1456221600
1456740000
1456826400
1457344800
1457427600
1457942400
1457427600
1457942400
1482832800
1483351200
1483437600
1483956000
1484042400
1484560800
1484647200
1485165600
1485252000
1485770400
1485856800
1486375200
1486461600
1486980000
1487066400
1487584800
1487671200
1488189600
1488276000
1488794400
1488276000
1488794400
1488877200
1489392000
1488877200
1489392000
1514282400
1514800800
1514887200
1515405600
1515492000
1516010400
1516096800
1516615200
1516701600
1517220000
1517306400
1517824800
1517911200
1518429600
1518516000
1519034400
1519034400

C:\Users\joshl\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:153: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


1405411200
1405929600
1406016000
1406534400
1406016000
1406534400
1431417600
1431936000
1431504000
1432022400
1432022400
1432540800
1432627200
1433145600
1433232000
1433750400
1433836800
1434355200
1434441600
1434960000
1435046400
1435564800
1435651200
1436169600


C:\Users\joshl\Anaconda3\lib\site-packages\psaw\PushshiftAPI.py:153: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


1436256000
1436774400
1436860800
1437379200
1437465600
1437984000
1437465600
1437984000
1463472000
1463990400
1464076800
1464595200
1464681600
1465200000
1464768000
1465286400
1465891200
1466409600
1466496000
1467014400
1467705600
1468224000
1468310400
1468828800
1468915200
1469433600
1469001600
1469520000
1469520000
1470038400
1469520000
1470038400
1494921600
1495440000
1495526400
1496044800
1496131200
1496649600
1497340800
1497859200
1497945600
1498464000
1498032000
1498550400
1499155200
1499673600
1499760000
1500278400
1500364800
1500883200
1500969600
1501488000
1501574400
1502092800
1526976000
1527494400
1527580800
1528099200
1528185600
1528704000
1528790400
1529308800
1529395200
1529913600
1530000000
1530518400
1530604800
1531123200
1531209600
1531728000
1531814400
1532332800
1532419200
1532937600
1533024000
1533542400


In [7]:
df.to_pickle('dfBachAndEtte5seasons.pickle')